![banner.png](banner.png)

<h2 style="color:#ffffff; background-color:#004aac; padding: 10px; text-align:left; border: 1px solid #004aac;">A. System Overview</h2>

This machine learning system is designed to showcase an end-to-end implementation covering the entire lifecycle of a machine learning model. It integrates various MLOps tools and technologies discussed during the DASCI 270 sessions, that includes facilitating data ingestion, preprocessing, training, validation, deployment, and monitoring of the model. The system is also structured to handle drift detection to ensure the model remains effective and accurate over time. This document will guide you through interacting with the deployed **Equipment Failure Prediction** system, including making predictions, retrieving model information, and checking data drift, as well as provide detailed documentation of each component and their functionalities.

<h2 style="color:#d7633a; padding: 5px; text-align:left; border: 1px solid #d7633a;">Components</h2>

1. **Machine Learning Model** - This system utilizes XGBoost, a highly efficient and scalable machine learning algorithm for classification tasks. It serves as the core predictive model in our system, specifically designed to predict equipment failure.

2. **Data Pipeline (Dagster)** - Orchestrates the workflow for data ingestion, preprocessing, and preparation. Dagster manages the sequence of these operations to ensure data flows correctly from one process to another, maintaining a clear and manageable execution order.

3. **Experiment Tracking (MLflow)** - Provides a framework to track experiments, including model training runs, parameters, metrics, and artifacts, enabling easier debugging and optimization. It stores models, performance metrics, and custom objects like drift reports, making them easy to access and compare across different runs.

4. **Model Serving (FastAPI)** - Deploys the trained model through a REST API using FastAPI, facilitating easy access to the model’s predictive capabilities. The API handles requests for predictions and provides model metadata, ensuring input validation and structured outputs.

5. **Containerization (Docker)** - Containerizes the services making up the system to ensure consistency and reproducibility across environments. Each isolated environment ("container") contains all necessary dependencies for each service, which can be easily deployed on any system supporting Docker.

6. **Drift Detection (Evidently AI)** - Integrates Evidently AI to monitor the model for any signs of data or concept drift. This component is crucial for maintaining the model's accuracy, providing insights into how the data characteristics and relationships are changing over time.

7. **Data Validation (Pydantic)** - Ensures that the data received by the API matches the expected format and type. This prevents errors during model prediction and ensures reliable model performance.

8. **Testing (Pytest and Github Actions)** - Uses Pytest to develop and run comprehensive tests that validate the correctness of the data processing and feature engineering components. GitHub Actions automates these tests, ensuring that all code integrations meet quality standards and function as expected.

<h2 style="color:#d7633a; padding: 5px; text-align:left; border: 1px solid #d7633a;">Dataset</h2>

This project utilized the AI4I 2020 Predictive Maintenance dataset, a synthetic dataset sourced from UCI Machine Learning Repository: https://archive.ics.uci.edu/dataset/601/ai4i+2020+predictive+maintenance+dataset. It contains 10,000 data points with the following columns/variables:

| Variables          | Type        | Description                                                                                         |
|--------------------|-------------|-----------------------------------------------------------------------------------------------------|
| UID                | Integer     | Unique identifier ranging from 1 to 10000                                                           |
| Product ID         | Categorical | Consists of a letter (L, M, H) for low, medium, and high product quality variants with a serial number |
| Type               | Categorical | Not specified                                                                                       |
| Air temperature    | Continuous  | Generated using a random walk process, normalized around 300 K with a standard deviation of 2 K      |
| Process temperature| Continuous  | Generated using a random walk process, normalized to a standard deviation of 1 K, plus air temperature + 10 K |
| Rotational speed   | Integer     | Calculated based on a power of 2860 W with normally distributed noise                               |
| Torque             | Continuous  | Normally distributed around 40 Nm with a standard deviation of 10 Nm, no negative values             |
| Tool wear          | Integer     | Tool wear added by quality variants H/M/L are 5/3/2 minutes respectively                             |
| Machine failure    | Integer     | Indicates if the machine has failed (1) or not (0) in this particular datapoint                      |
| TWF                | Integer     | Target feature indicating if a specific type of failure occurred or not (1/0)                        |


<h2 style="color:#ffffff; background-color:#004aac; padding: 10px; text-align:left; border: 1px solid #004aac;">B. Setup Instructions</h2>

This section provides detailed instructions on how to set up and run the system using Docker Compose. The instructions assume you have Docker and Docker Compose installed on your machine. If not, please install them from the [Docker official website](https://www.docker.com/) before proceeding.

<h2 style="color:#d7633a; padding: 5px; text-align:left; border: 1px solid #d7633a;">1. Clone the Repository</h2>

Begin by cloning the project repository from GitHub. This will retrieve all the necessary code and configuration files needed to set up and run the project locally. Execute the following commands in your terminal:

```bash
# Clone the repository
git clone https://github.com/raymundojavajr/ml-fp.git

# Change directory to the cloned repository
cd ml-fp
```

These commands clone the repository into a directory named `ml-fp` on your local machine and then change your current directory to `ml-fp`, placing you in the project's root directory where subsequent setup steps are performed.

<h2 style="color:#d7633a; padding: 5px; text-align:left; border: 1px solid #d7633a;">2. Configure Environment Variables</h2>

To manage environment variables, create a `.env` file in the root directory of the project. This file will store all necessary environment settings. Ensure this file is not tracked by version control by adding it to your `.gitignore` file.

For local development, you'll also need to set up a virtual environment to isolate your project's dependencies from the global Python environment. Here's how to set up and activate a virtual environment using Python's built-in `venv` module:

```bash
# Create a virtual environment
python -m venv venv

# Activate the virtual environment
# On Windows, use:
venv\Scripts\activate

# On Unix or MacOS, use:
source venv/bin/activate
```

If you are using uv as your package manager, ensure you have it installed and then synchronize your environment:

```bash
uv sync
```

<h2 style="color:#d7633a; padding: 5px; text-align:left; border: 1px solid #d7633a;">3. Install Dependencies/Packages</h2>

To ensure all functionalities of the pipeline work seamlessly, it is crucial to install the necessary Python packages. You can install dependencies using the `requirements.txt` for traditional pip installations or use `uv sync` which handles dependencies listed in `pyproject.toml`.

**Using requirements.txt**
```bash
# Install dependencies from requirements.txt
pip install -r requirements.txt
```

**Using uv for pyproject.toml**
If you are using `uv` as your package manager, you can synchronize your environment directly:

```bash
# Sync environment and install dependencies with uv
uv sync
```

**Verify Installation**
After installing the necessary packages, verify that all packages are correctly installed and configured to support all functionalities of the pipeline:

```bash
# List installed packages to verify installation
pip list
```

This command displays all installed Python packages in your environment, allowing you to check that all required dependencies are correctly installed. This step is crucial for catching any missing or incorrectly installed packages before proceeding with further setup or development.

#### **List of Libraries used in the pipeline:**

The following libraries are integral to the pipeline, supporting various aspects of machine learning model development, deployment, and maintenance:

**Core Libraries**

- mlflow - Tracks experiments, including model training runs, parameters, and metrics.
- pandas - Data manipulation and analysis.
- matplotlib - Used for creating static, interactive, and animated visualizations in Python.
- joblib - Efficient saving of models and other large data structures.
- requests - Handles HTTP requests to external services or APIs.

**Machine Learning Models and Tools**

- xgboost - Implements machine learning algorithms under the Gradient Boosting framework.
- scikit-learn - Tools for data mining and data analysis, including model training and evaluation functions.
- shap - Explains the output of machine learning models.
- evidently - Monitors model performance and detects data drift.

**Dagster Integration**

- dagster, dagit, dagster-postgres, dagster-pandas, dagster-mlflow - Provides orchestration of data workflows, asset management, and integration with other ML tools.

**API Development and Deployment**

- fastapi - Modern, fast (high-performance) web framework for building APIs.
- uvicorn - ASGI server for FastAPI, which allows async programming.
- pydantic - Data validation by defining data types and structures for FastAPI endpoints.
- python-dotenv - Reads key-value pairs from a .env file and sets them as environment variables.

**Utilities and Miscellaneous**

- matplotlib - For plotting and visualizing data in various forms.
- pydantic - For data validation and settings management via data models.

<h2 style="color:#ffffff; background-color:#004aac; padding: 10px; text-align:left; border: 1px solid #004aac;">C. Orchestrating ML Workflows with Dagster</h2>

This section explains the execution of the **Machine Failure Prediction** pipeline, which is fully managed within Dagster. This includes setting up and running the pipeline components via the Dagster UI.

#### **Pipeline Overview**

The pipeline is designed to handle various stages from data processing to model deployment, ensuring thorough analysis and prediction of machine failures.

#### **Dagster Assets in the Pipeline**

- **Data Ingestion (Upstream Assets):**
  - `download_machine_data` retrieves raw sensor and operation data from sources.
  - `raw_machine_data` stores the fetched data for subsequent processing.
- **Data Preparation (Downstream from Raw Data):**
  - `cleaned_machine_data` performs data cleaning and preprocessing to prepare for analysis.
- **Model Development (Downstream from Prepared Data):**
  - `train_test_split_data` segments the data into training and testing subsets.
  - `feature_preprocessor` applies necessary transformations for the predictive model.
  - `trained_model` encompasses the training of the failure prediction model.
- **Operational Deployment (Downstream from Model Development):**
  - `model_evaluation` assesses the predictive accuracy and robustness of the model.
  - `saved_model` handles the storage of the validated model.
- **Continuous Improvement (Downstream from Deployment):**
  - `drift_detection_data` simulates potential drift in machine operation data.
  - `cleaned_drift_data` prepares this data for drift analysis.
  - `preprocessed_drift_data` further processes the data to fit analysis models.
  - `drift_reports` creates detailed reports on any detected drift, helping in proactive maintenance.

#### **Executing the Pipeline**

To operationalize all assets in the **Machine Failure Prediction** pipeline, navigate to the **Dagster UI** at (`http://localhost:3000`) and follow these steps:

- **Start with Upstream Assets:** Initiate the pipeline by materializing assets such as `download_machine_data` and `raw_machine_data`.
- **Advance through the Pipeline:** Continue activating downstream assets in sequence to ensure a logical flow of data processing and model training.
- **Finalize with Evaluation and Monitoring:** Complete the execution by materializing assets related to model evaluation and drift detection, which are crucial for maintaining the accuracy and relevance of the prediction model.

This structured approach ensures that each component of the pipeline functions efficiently and that dependencies are correctly managed from data collection through to drift detection.

<h2 style="color:#ffffff; background-color:#004aac; padding: 10px; text-align:left; border: 1px solid #004aac;">D. Deploying the Full ML Pipeline Stack with Docker Compose</h2>

This project leverages Docker Compose for containerization, ensuring cohesive and efficient functioning of all components.

#### **Containerized Services in `docker-compose.yaml`**

Below are the services configured in the `docker-compose.yaml` file, outlining their roles and the ports they operate on:

| **Service** | **Purpose** | **Port** |
|-------------|-------------|---------|
| **PostgreSQL (`postgres_service`)** | Database for storing pipeline data | Internal |
| **MinIO (`minio_service`)** | S3-compatible storage for artifacts | `9000`, `9001` |
| **MLflow Tracking Server (`mlflow_service`)** | Manages experiment tracking and artifact logging | `5000` |
| **FastAPI Model Server (`fastapi_service`)** | Hosts the machine failure prediction model | `8000` |
| **Dagster Server (`dagster_service`)** | Orchestrates and monitors the pipeline | `3000` |

#### **Step 1: Launch All Services**
Execute the following command to build and initiate all services:

```bash
docker-compose up --build -d
```

This command:
- Constructs any necessary Docker images.
- Launches all services in detached mode (`-d`).

#### **Step 2: Verify Service Containers**
To confirm that all Docker containers are active, run:

```bash
docker ps
```

#### **Step 3: Service Access Points**
Here are the URLs to access each service's user interface:

| **Service**               | **URL**                                 |
|---------------------------|-----------------------------------------|
| **Dagster UI**            | [http://localhost:3000](http://localhost:3000) |
| **MLflow Tracking UI**    | [http://localhost:5000](http://localhost:5000) |
| **FastAPI Model Server**  | [http://localhost:8000](http://localhost:8000) |
| **MinIO Console**         | [http://localhost:9001](http://localhost:9001) |

#### **Step 4: Configure MinIO for MLflow**
Prior to using Dagster for asset materialization, configure a storage bucket in MinIO:

1. Visit [http://localhost:9001](http://localhost:9001)
2. Log in using:
   - **Username:** `minio_user`
   - **Password:** `minio_password`
3. Establish a new bucket named `mlflow`.
4. Amend your `.env` file with the MinIO Access Keys obtained:

   ```ini
   # MinIO Access Keys
   MINIO_ACCESS_KEY=your_generated_access_key
   MINIO_SECRET_ACCESS_KEY=your_generated_secret_key
   ```

#### **Step 5: Materialize Assets in Dagster**
With all services operational, proceed to materialize assets:

- Navigate to the [Dagster UI](http://localhost:3000).
- Select "Assets" → "Materialize Selected" to execute the pipeline.

#### **Service Management Commands**

| **Action**                | **Command**                          |
|---------------------------|--------------------------------------|
| **Restart all services**  | `docker-compose down && docker-compose up -d` |
| **Stop all services**     | `docker-compose down`                |

<h2 style="color:#ffffff; background-color:#004aac; padding: 10px; text-align:left; border: 1px solid #004aac;">E. Happy Path</h2>

<h2 style="color:#d7633a; padding: 5px; text-align:left; border: 1px solid #d7633a;">Prediction Request</h2>

In this section, we demonstrate how to use the FastAPI model server for predictions. We'll send a set of equipment parameters and status data to receive predictions on potential machine failures.

- **Input Data:** This includes a series of variables such as equipment type, air and process temperatures, rotational speed, torque, tool wear, and previous instances of machine failure.
- **API Endpoint:** This is the URL where the FastAPI server accepts prediction requests.
- **Response Handling:** Upon receiving the prediction request, the server provides a forecast indicating whether a machine is likely to fail. This prediction is displayed in the output.

Execute the following cell to initiate a prediction request.

In [16]:
import pandas as pd
import requests

# Load test data
csv_path = "synthetic_data.csv"
test_df = pd.read_csv(csv_path)
json_data = test_df.to_dict(orient="records")

# Define FastAPI Prediction Endpoint
predict_url = "http://localhost:8000/predict"

# Send Prediction Request
predict_response = requests.post(predict_url, json=json_data)

# Handle Response
print("\n===== Prediction Response =====")
print("Status Code:", predict_response.status_code)

try:
    response_json = predict_response.json()
    
    # Extract predictions safely
    predictions = response_json.get("predictions", [])

    if predictions:
        print(f"Showing first 20 of {len(predictions)} predictions:")
        for i, pred in enumerate(predictions[:20]):
            print(f"  Row {i+1}: {pred}")
    else:
        print("No predictions returned.")

except requests.exceptions.JSONDecodeError:
    print("Invalid response received from Prediction API.")


===== Prediction Response =====
Status Code: 200
Showing first 20 of 1000 predictions:
  Row 1: 0
  Row 2: 0
  Row 3: 1
  Row 4: 0
  Row 5: 0
  Row 6: 0
  Row 7: 0
  Row 8: 1
  Row 9: 1
  Row 10: 0
  Row 11: 1
  Row 12: 0
  Row 13: 0
  Row 14: 0
  Row 15: 1
  Row 16: 0
  Row 17: 1
  Row 18: 1
  Row 19: 0
  Row 20: 1


#### Response Interpretation

The response indicates that the request was successfully processed, as shown by the **Status Code: 200**. This confirms that the FastAPI service is running properly and returning predictions without any errors. The model processed **1,000 rows** from the test dataset, but only the first 20 predictions are displayed for reference.  

Each prediction is represented as either **`0`** or **`1`**, where **`0`** likely indicates normal operation with no failure detected, while **`1`** suggests a potential failure that may require maintenance. For example, Row 1 has a prediction of **`0`**, meaning no failure was detected, while Row 3 has a prediction of **`1`**, indicating a possible failure. Similar patterns are observed in Rows 8, 11, and 15, where failure predictions appear sporadically.  

The model does not predict failures in a uniform manner, suggesting that different conditions in the dataset contribute to varying predictions. This pattern aligns with real-world predictive maintenance scenarios, where failures occur intermittently rather than consistently across all cases.  

### **Next Steps**  

To gain deeper insights, it is important to analyze cases where failures (`1`) are predicted and identify common factors such as high tool wear, extreme temperatures, or unusual operational conditions. Additionally, running a **drift detection analysis** can help determine if the dataset has significantly changed over time, which could impact the model’s accuracy. Finally, evaluating the model’s **performance metrics** (such as accuracy, F1-score, and recall) will help ensure that the predictions remain reliable and actionable.

<h2 style="color:#d7633a; padding: 5px; text-align:left; border: 1px solid #d7633a;">Model Information Retrieval</h2>

To effectively interact with our FastAPI model server, it's essential to retrieve the model's configuration. This process involves accessing detailed metadata from the /model endpoint, which helps users understand the underlying model's structure and operational parameters.

**Retrieve Model Information**

The model server offers detailed metadata which includes:

* **Input schema** detailing the required format for prediction inputs.
* **Training Hyperparameters** used during the training process.
* **Important features** that significantly impact the prediction outcomes.

Execute the following cell to obtain detailed information about the model hosted on FastAPI:

In [ ]:
import joblib

# Define the model path
model_path = "C:/Users/karth/Documents/MSDS/ml-fp/src/models/trained_model.pkl"

# Load the model
model = joblib.load(model_path)
print("\nModel loaded successfully")
print(f"Model Type: {type(model)}")

# Retrieve Model Hyperparameters
params = model.get_xgb_params()
print("\n===== Model Hyperparameters =====")
for param, value in params.items():
    print(f"  {param}: {value}")

# Retrieve Feature Importance
if hasattr(model, "feature_importances_") and hasattr(model, "feature_names_in_"):
    feature_importance = {feature: importance for feature, importance in zip(model.feature_names_in_, model.feature_importances_)}
    print("\n===== Feature Importances =====")
    for feature, importance in feature_importance.items():
        print(f"  {feature}: {importance:.4f}")
else:
    print("\nFeature importance is not available.")

# Verify Model Training
if hasattr(model, "n_features_in_"):
    print(f"\nModel trained on {model.n_features_in_} features")
else:
    print("\nModel might not be properly trained")



✅ Model loaded successfully!
📌 Model Type: <class 'xgboost.sklearn.XGBClassifier'>

===== 🔧 Model Hyperparameters =====
  objective: binary:logistic
  base_score: None
  booster: None
  colsample_bylevel: None
  colsample_bynode: None
  colsample_bytree: None
  device: None
  eval_metric: logloss
  gamma: None
  grow_policy: None
  interaction_constraints: None
  learning_rate: None
  max_bin: None
  max_cat_threshold: None
  max_cat_to_onehot: None
  max_delta_step: None
  max_depth: None
  max_leaves: None
  min_child_weight: None
  monotone_constraints: None
  multi_strategy: None
  n_jobs: None
  num_parallel_tree: None
  random_state: 42
  reg_alpha: None
  reg_lambda: None
  sampling_method: None
  scale_pos_weight: None
  subsample: None
  tree_method: None
  validate_parameters: None
  verbosity: None

===== 🔥 Feature Importances =====
  UDI: 0.0086
  Air_temperature_K: 0.0113
  Process_temperature_K: 0.0085
  Rotational_speed_rpm: 0.0105
  Torque_Nm: 0.0080
  Tool_wear_min: 0

#### Insights on Retrieved Model Information

### **Explanation of Your Model Output**

####  **Model Type:** `<class 'xgboost.sklearn.XGBClassifier'>`
- This means the model is an **XGBoost Classifier** (used for classification tasks).
- It's designed to predict a categorical outcome (e.g., binary classification of failure detection in predictive maintenance).

---

## **Model Hyperparameters**
These are the **settings** used when training the model.

| **Hyperparameter**      | **Value**        | **Explanation** |
|-------------------------|-----------------|-----------------|
| **objective**          | `binary:logistic` | Model predicts probabilities for a binary classification problem (e.g., failure or no failure). |
| **eval_metric**        | `logloss`        | The model was trained using **logarithmic loss**, a standard metric for classification. |
| **random_state**       | `42`             | Ensures reproducibility of results by fixing the randomness. |
| Other parameters       | `None`           | These hyperparameters were not explicitly set and used default values. |

 **Issue: Many hyperparameters are `None`**
- This suggests that either they were not properly logged or the saved model did not retain all training details.
- It might still work, but if you want a fully configured model, ensure **all necessary hyperparameters are explicitly set** when training.

---

## **Feature Importances**
These indicate **how much each feature contributes to the model's decisions**.

| **Feature**             | **Importance** | **Explanation** |
|-------------------------|--------------|-----------------|
| **UDI**                 | `0.0086`      | Has very little impact on predictions. |
| **Air_temperature_K**   | `0.0113`      | Slightly more important, but still low. |
| **Process_temperature_K** | `0.0085`    | Low impact. |
| **Rotational_speed_rpm** | `0.0105`    | Low impact. |
| **Torque_Nm**           | `0.0080`      | Low impact. |
| **Tool_wear_min**       | `0.0211`      | More important than previous features. |
| **Type_encoded**        | `0.0000`      | **Not contributing at all.** Might be irrelevant. |
| **Product_ID_encoded**  | `0.0069`      | Very low importance. |
| **Failure_Type_encoded** | **`0.9251`** | 🚨 **Dominates the model.** Almost all decisions are based on this feature. |

🔍 **Insight:** 
- The model **heavily relies on `Failure_Type_encoded` (92.51%)**, meaning it may not be using other features effectively.
- If this feature is incorrectly encoded or biased, the model may **overfit** and generalize poorly.
- Consider **retraining the model with proper feature engineering** to balance importance.

---

### **Model trained on 9 features**
- The model recognizes **9 input features** from the dataset.
- All of them are considered during training, but as seen in feature importance, some contribute **far more than others**.





<h2 style="color:#ffffff; background-color:#004aac; padding: 10px; text-align:left; border: 1px solid #004aac;">F. Drift Detection Demonstration</h2>

 To use the Drift Detection API, first, ensure that the Drift Monitoring Service is running by starting it with Docker using the command docker-compose up -d drift_monitoring_service. Next, prepare your input data in the correct format, ensuring it is structured as a list of dictionaries before making the request. Once the data is ready, execute the provided script in a Python environment such as Jupyter Notebook or a standalone script. After running the script, check the response. If the status code is 200, the request was successfully processed, and you can review the drift detection results. If drift is detected (True), it indicates significant changes in the input data compared to the reference dataset. If you encounter an invalid response, check the service logs by running docker logs drift_monitoring_service --tail 50 to diagnose potential issues.

In [17]:


# Define Drift Detection Endpoint
drift_url = "http://localhost:8085/drift"

# Send Drift Detection Request
drift_response = requests.post(drift_url, json=json_data)

# Handle Response
print("\n===== Drift Detection Response =====")
print("Status Code:", drift_response.status_code)

try:
    print("Response:", drift_response.json())
except requests.exceptions.JSONDecodeError:
    print("❌ Drift Monitoring Service returned an invalid response.")



===== Drift Detection Response =====
Status Code: 200
Response: {'drift_detected': True, 'drift_score': 0.0, 'drift_report_html_path': 'mlflow_artifacts/drift_report.html'}


**Note:**

 The drift detection response indicates that drift has been detected in the dataset, as shown by "drift_detected": True. However, the drift score is 0.0, which suggests that while drift is flagged, no significant changes in feature distributions were quantified. This may be due to an issue with how the drift score is computed or an edge case in the data. The detailed drift analysis report has been saved as an HTML file at mlflow_artifacts/drift_report.html, which can be reviewed for a deeper understanding of which features have drifted and the extent of the changes.

<h2 style="color:#d7633a; padding: 5px; text-align:left; border: 1px solid #d7633a;">Accessing Evidently AI from MLflow to Generate Drift Reports</h2>

Once the `drift_reports` asset is activated within **Dagster UI**, these reports are subsequently recorded in **MLflow**, specifically under Experiments → Artifacts.

**How to Access Drift Reports in MLflow**

1. Launch the **MLflow UI** by visiting http://localhost:5000.


2. Proceed to **Experiments** and select the **Most Recent Run**.
3. Navigate to the **Artifacts** tab.
4. Find and download the `drift_report.html` to examine the comprehensive drift analysis. 

![Drift.png](Drift.png)

<h2 style="color:#d7633a; padding: 5px; text-align:left; border: 1px solid #d7633a;">Drift Result Analysis</h2>



The drift detection report reveals that significant changes have occurred in the dataset, indicating that the distribution of new data differs considerably from the historical reference data. The analysis examined nine features, all of which exhibited drift, meaning that 100% of the dataset has changed in some way. Since the dataset drift flag is marked as `true`, this suggests that the model may not perform as expected on the new data, potentially leading to unreliable predictions.

Several key features have undergone notable drift. For example, **air temperature** has a drift score of `0.6975`, indicating a significant shift in distribution. Similarly, **process temperature** (`0.7437`) and **rotational speed** (`0.5114`) show noticeable variations compared to the reference data. Additionally, **tool wear (`1.1567`) and failure type encoding (`1.4140`)** have experienced high levels of drift, suggesting that the nature of failures in the dataset may have changed over time. The **product ID encoding (`0.7212`) and torque (`0.2536`)** have also drifted, meaning that machine and operational characteristics may no longer match historical patterns.

One of the most concerning drifts is seen in the **Unique Device Identifier (UDI) with a drift score of `1.5588`**, implying that the distribution of machines or components being monitored is significantly different from what the model was originally trained on. Additionally, the encoding of machine types has drifted (`0.2244`), indicating possible changes in the categories of equipment being processed.

Given that all features have experienced drift, this suggests a fundamental shift in the dataset, which may impact the reliability of predictions. If this drift is due to changes in the operational environment, machine wear, or sensor readings, the model may need to be retrained with updated data to maintain accuracy. Further investigation is required to determine whether the drift is temporary or a sign of a long-term change in the system. Reviewing the **drift report in detail** and comparing it with model performance metrics will help decide if retraining is necessary.

<h2 style="color:#ffffff; background-color:#004aac; padding: 10px; text-align:left; border: 1px solid #004aac;">F. Reproducibility</h2>

This pipeline is designed to be fully reproducible to facilitate reliable and consistent results in machine learning model development, testing, and deployment processes.

#### **Steps to Ensure Reproducibility**

1. **Running Services:** Make sure that FastAPI and MLflow services are active. These are essential for handling API requests and tracking experiments, respectively.

    ```plaintext
    FastAPI should be accessible at: http://localhost:8000
    MLflow should be accessible at: http://localhost:5000
    ```

2. **Model Artifacts and Logs:** Confirm that the latest model artifacts and corresponding logs are properly generated and accessible in MLflow. This ensures that the models being deployed and tested are the latest iterations.

3.  **Configuration and Environment Variables:** Check that all environment variables and configurations are set correctly. This includes database URIs, API keys, and other service-specific configurations that might affect the system's behavior.

<h2 style="color:#ffffff; background-color:#004aac; padding: 10px; text-align:left; border: 1px solid #004aac;">G. References</h2>

[1] [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/601/ai4i+2020+predictive+maintenance+dataset)

[2] [Dagster Documentation](https://docs.dagster.io/)

[3] [MLflow Documentation](https://www.mlflow.org/docs/2.1.1/index.html?utm_source=google&utm_medium=cpc&utm_term=&utm_campaign=&gad_source=1&gclid=CjwKCAjwp8--BhBREiwAj7og1-SMU90CS2ghrTsi-kHfz67v2JAaQR5QQfRzhIKUK8VywBypvMULGxoCqhoQAvD_BwE)

[4] [FastAPI Documentation](https://fastapi.tiangolo.com/)

[5] [Docker Documentation](https://docs.docker.com/)

[6] [Evidently AI Documentation](https://docs.evidentlyai.com/introduction)